In [1]:
!pip install catboost
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 12.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from __future__ import division
import matplotlib.pyplot as plt
import copy
import os
import pickle
import datetime
import glob
import joblib
import optuna
from zipfile import ZipFile
from datetime import datetime
import urllib.request
from optuna.integration import LightGBMPruningCallback
from math import sqrt
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from scipy.stats import entropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV,cross_val_score,KFold
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool, metrics, cv,CatBoostRegressor
#import category_encoders as ce
import scipy as sp
import lightgbm as lgbm
import xgboost as xgb

In [3]:
from google.colab import drive
drive.mount('/drive', force_remount=True)

Mounted at /drive


In [4]:
base_path_train = '/drive/My Drive/recsys23/train'
base_path_test = '/drive/My Drive/recsys23/test'
base_path_trainfiles = '/drive/My Drive/recsys23/train_files'
base_path_preds = '/drive/My Drive/recsys23/preds'

In [5]:
# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(base_path_train) if f.endswith('.csv')]

# Create an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Loop through each CSV file and append its data to the combined DataFrame
for csv_file in csv_files:
    file_path = os.path.join(base_path_train, csv_file)
    df = pd.read_csv(file_path,sep = '\t')
    combined_df = combined_df.append(df, ignore_index=True)

# Print the combined DataFrame
print(combined_df.head())

<ipython-input-5-c571df68d2b8>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
<ipython-input-5-c571df68d2b8>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
<ipython-input-5-c571df68d2b8>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
<ipython-input-5-c571df68d2b8>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
<ipython-input-5-c571df68d2b8>:11: FutureWarning: The frame.append method is deprecated and will

       f_0  f_1    f_2    f_3    f_4    f_5    f_6    f_7    f_8    f_9  ...  \
0  2541207   50  20095    563  31686  15908    590  27941  21218  21533  ...   
1  2541533   53  17769  22294   5503  25604  28739  27941  21218  14659  ...   
2  2541992   66  26325   7779  21563  15908    328  27941  21218  14659  ...   
3  2542108   46  14963  22294  26466  25604  12305  27941  21218   9638  ...   
4  2542137   51  17758  22294   4870  25604  11727  27941  21218  31372  ...   

       f_72  f_73      f_74      f_75      f_76  f_77       f_78  f_79  \
0  0.000000   0.0  0.077128  1.156922  0.269948   0.0   0.000000   0.0   
1  0.571121   0.0  0.038564  1.156922  0.269948   0.0   0.000000   0.0   
2  0.000000   0.0  0.000000  0.077128  0.000000   0.0   0.000000   0.0   
3  0.000000   0.0  0.115692  1.156922  0.269948   0.0   0.000000   0.0   
4  1.142243   0.0  0.115692  1.156922  0.269948   0.0  37.384575   0.0   

   is_clicked  is_installed  
0           0             0  
1           0 

In [6]:
y_click = combined_df[['is_clicked']]
y_install = combined_df[['is_installed']]
full_train = combined_df.drop(['is_clicked','is_installed','f_0','f_1'],axis=1)

In [7]:
full_train_cat = full_train.copy()
full_train_cat['f_30'] = full_train_cat['f_30'].fillna(-1).astype(int).replace(-1, 'Null')
full_train_cat['f_31'] = full_train_cat['f_31'].fillna(-1).astype(int).replace(-1, 'Null')

In [8]:
GPU_ENABLED = True
cat_cols = []
for i in range(2,33):
  cat_cols.append("f_{0}".format(str(i)))

In [9]:
# cat install params
best_param_cat_fix_cv = {'learning_rate': 0.06314591534089734, 'reg_lambda': 73.0979976315868, 'subsample': 0.5918440238472964,
                         'random_strength': 26.555921799287248, 'depth': 8, 'min_data_in_leaf': 9, 'leaf_estimation_iterations': 4,'iterations':55}

In [10]:
best_param_lgbm_fix_install = {'scale_pos_weight': 1, 'reg_alpha': 0.11958415677338752, 'reg_lambda': 2.222708594529026, 'colsample_bytree': 0.3,
                               'subsample': 0.6, 'learning_rate': 0.008, 'max_depth': 70, 'num_leaves': 845, 'min_child_samples': 12,
                               'min_data_per_groups': 52,'eval_metric': 'logloss','objective' : 'binary','random_state': 48,'n_estimators': 3000}

In [ ]:
model_best_lgbm_inst = lgbm.LGBMClassifier(**best_param_lgbm_fix_install)
model_best_lgbm_inst.fit(full_train,y_install,verbose=False,categorical_feature=cat_cols)

In [11]:
cat_model_inst = CatBoostClassifier(
        loss_function="Logloss",
        eval_metric = 'Logloss',
        random_state=42,
        **best_param_cat_fix_cv,
    )

In [ ]:
cat_model_inst.fit(full_train_cat, y_install, cat_features=cat_cols)

In [17]:
best_param_lgbm_fix_click = {'scale_pos_weight': 1, 'reg_alpha': 0.46636269130289787, 'reg_lambda': 0.521102666434752,
 'colsample_bytree': 0.3, 'subsample': 0.8, 'learning_rate': 0.004, 'max_depth': 8,
 'num_leaves': 454, 'min_child_samples': 300, 'min_data_per_groups': 81,'n_estimators': 20000}

In [18]:
# this is click params
best_param_cat_fix_cv_val_click = {'learning_rate': 0.17484326779116222, 'reg_lambda': 0.6277515640834546,
 'subsample': 0.6670431730949343, 'random_strength': 17.198749834468376,
 'depth': 11, 'min_data_in_leaf': 20, 'leaf_estimation_iterations': 15,'iterations':12}

In [ ]:
model_best_lgbm_click = lgbm.LGBMClassifier(**best_param_lgbm_fix_click)
model_best_lgbm_click.fit(full_train,y_click,verbose=False,categorical_feature=cat_cols)

In [19]:
cat_model_click = CatBoostClassifier(
        loss_function="Logloss",
        eval_metric = 'Logloss',
        random_state=42,
        **best_param_cat_fix_cv_val_click,
    )

In [ ]:
cat_model_click.fit(full_train_cat, y_click, cat_features=cat_cols)

Test Set

In [21]:
# Get a list of all CSV files in the folder
csv_files_test = [f for f in os.listdir(base_path_test) if f.endswith('.csv')]

# Create an empty DataFrame to store the combined data
combined_df_test = pd.DataFrame()

# Loop through each CSV file and append its data to the combined DataFrame
for csv_file in csv_files_test:
    file_path = os.path.join(base_path_test, csv_file)
    df = pd.read_csv(file_path,sep = '\t')
    combined_df_test = combined_df_test.append(df, ignore_index=True)

# Print the combined DataFrame
print(combined_df_test.head())

     f_0  f_1    f_2    f_3    f_4    f_5    f_6    f_7    f_8    f_9  ...  \
0  64505   67  26325   7152  21563  19475  31440  27941  21621  14659  ...   
1  64506   67  20095    563  22861  19475  21280  27941  19203  14659  ...   
2  64507   67    890  22294  18294  21545  20210  27941  18800   9638  ...   
3  64508   67  20095    563  22861  25604  21280  27941  18800  14659  ...   
4  64509   67  27426  22294  11338  19475  23855  27941  21218   9638  ...   

       f_70  f_71      f_72  f_73      f_74      f_75      f_76  f_77  f_78  \
0  1.519085   0.0  0.000000   0.0  0.000000  0.000000  0.000000   0.0   0.0   
1  0.000000   0.0  0.000000   0.0  0.115692  1.156922  0.269948   0.0   0.0   
2  0.982995   0.0  0.000000   0.0  0.000000  1.156922  0.269948   0.0   0.0   
3  0.000000   0.0  2.284486   0.0  0.000000  1.156922  0.269948   0.0   0.0   
4  0.986040   0.0  0.000000   0.0  0.077128  0.077128  0.077128   0.0   0.0   

   f_79  
0   0.0  
1   0.0  
2   0.0  
3   0.0  
4   0.

<ipython-input-21-02c38ca86c48>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df_test = combined_df_test.append(df, ignore_index=True)


In [22]:
combined_df_test = combined_df_test.drop(['f_0','f_1'],axis=1)

In [23]:
combined_df_test.head()

,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,f_11,...,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79
0,26325,7152,21563,19475,31440,27941,21621,14659,21574,31460,...,1.519085,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,20095,563,22861,19475,21280,27941,19203,14659,21574,27957,...,0.000000,0.0,0.000000,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0
2,890,22294,18294,21545,20210,27941,18800,9638,19343,30375,...,0.982995,0.0,0.000000,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
3,20095,563,22861,25604,21280,27941,18800,14659,22970,31863,...,0.000000,0.0,2.284486,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
4,27426,22294,11338,19475,23855,27941,21218,9638,21574,30778,...,0.986040,0.0,0.000000,0.0,0.077128,0.077128,0.077128,0.0,0.0,0.0


In [24]:
combined_df_test_cat = combined_df_test.copy()
combined_df_test_cat['f_30'] = combined_df_test_cat['f_30'].fillna(-1).astype(int).replace(-1, 'Null')
combined_df_test_cat['f_31'] = combined_df_test_cat['f_31'].fillna(-1).astype(int).replace(-1, 'Null')

In [ ]:
test_prob = cat_model_inst.predict_proba(combined_df_test_cat)
test_prob_pred_install_cat = pd.DataFrame()
test_prob_pred_install_cat['is_installed'] = test_prob[:,1]
test_prob_pred_install_cat.head()

In [ ]:
test_prob = model_best_lgbm_inst.predict_proba(combined_df_test)
test_prob_pred_install_lgb = pd.DataFrame()
test_prob_pred_install_lgb['is_installed'] = test_prob[:,1]
test_prob_pred_install_lgb.head()

In [ ]:
test_prob = cat_model_click.predict_proba(combined_df_test_cat)
test_prob_pred_click_cat = pd.DataFrame()
test_prob_pred_click_cat['is_clicked'] = test_prob[:,1]
test_prob_pred_click_cat.head()

In [ ]:
test_prob = model_best_lgbm_click.predict_proba(combined_df_test)
test_prob_pred_click_lgb = pd.DataFrame()
test_prob_pred_click_lgb['is_clicked'] = test_prob[:,1]
test_prob_pred_click_lgb.head()

In [ ]:
sub_df = pd.DataFrame()
sub_df['RowId'] = combined_df_test['f_0']
sub_df['is_clicked_cat'] = test_prob_pred_click_cat['is_clicked']
sub_df['is_clicked_lgb'] = test_prob_pred_click_lgb['is_clicked']
sub_df['is_clicked'] = sub_df[['is_clicked_lgb', 'is_clicked_cat']].mean(axis=1)
sub_df['is_installed_cat'] = test_prob_pred_install_cat['is_installed']
sub_df['is_installed_lgb'] = test_prob_pred_install_lgb['is_installed']
sub_df['is_installed'] = sub_df[['is_installed_lgb', 'is_installed_cat']].mean(axis=1)
sub_df.drop(['is_clicked_lgb','is_clicked_cat','is_installed_cat','is_installed_lgb'],axis=1,inplace=True)
sub_df.head()

In [ ]:
sub_df.to_csv(base_path_preds + '/sub.csv',sep='\t',index=False)